# Analysis of the AMLTK Pipeline
OpenFE-like Analysis

### To Do for the User

In [158]:
# TODO 1 for the user: Adapt list of labels (has to correspond to the method names for which the AMLTK was executed in the src/amtlk/main.py main()-method
label_list = ['Original Data', 'Self-feature-engineered Data', 'autofeat Data', 'OpenFE Data', 'H2O', 'MLJAR', 'AutoGluon']
# TODO 2 for the user: Adjust part size number according to the max_trials variable defined in src/amltk/main.py
part_size = 10
# TODO 3 for the user: Define the names of the used datasets
# all 10 datasets
datasets = ["abalone dataset", "moneyball dataset", "australian dataset", "blood transfusion service center dataset", "car dataset", "churn dataset", "credit g dataset", "kr-vs-kp dataset", "pc4 dataset", "wilt dataset"]#, "wine-quality-white dataset"]
# 5 small datasets
# datasets = ["australian dataset", "blood transfusion service center dataset", "car dataset", "credit g dataset", "wilt dataset"]
# 1 dataset to test new methods
# datasets = ['blood transfusion service center dataset']
# TODO 4 for the user. Define number of folds
folds = 1

### Imports


In [159]:
import pandas as pd
import numpy as np
from tabulate import tabulate

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

### Read Files

In [160]:
df_all = pd.DataFrame(columns=['Dataset', 'Type', 'Fold', 'Value'])

# Process the data
for dataset in datasets:
    for fold in range(folds):
        dataset_cleaned = dataset.replace(" ", "_").replace("-", "_")
        df_fold = pd.read_parquet(f"results_{dataset_cleaned}_fold_{fold}.parquet")

        for i, label in enumerate(label_list):
            metric_accuracy = df_fold["metric:accuracy [0.0, 1.0] (maximize)"].tolist()
            metric_accuracy = [sum(metric_accuracy[x:x+part_size]) for x in range(0, len(metric_accuracy), part_size)]

            # Only add value if lengths match
            if len(metric_accuracy) == len(label_list):
                value = metric_accuracy[i]
                df_all = df_all._append({'Dataset': dataset, 'Type': label, 'Fold': fold, 'Value': value}, ignore_index=True)

# Summarize by grouping and pivot the DataFrame
df_summary = df_all.groupby(['Dataset', 'Type'])['Value'].sum().reset_index()
pivot_table = df_summary.pivot(index='Dataset', columns='Type', values='Value').fillna(0)

pivot_table.reset_index(inplace=True)
pivot_table.to_parquet("tabular_data.parquet")

# Print the tabulated format using tabulate
print(tabulate(pivot_table, headers='keys', tablefmt='pretty'))

+---+------------------------------------------+-------------------+--------------------+-------------------+--------------------+-------------------+------------------------------+-------------------+
|   |                 Dataset                  |     AutoGluon     |        H2O         |       MLJAR       |    OpenFE Data     |   Original Data   | Self-feature-engineered Data |   autofeat Data   |
+---+------------------------------------------+-------------------+--------------------+-------------------+--------------------+-------------------+------------------------------+-------------------+
| 0 |             abalone dataset              | 0.472740779385746 | 2.4448526743183776 | 2.517729551331489 | 2.4522926779476477 | 2.539019189765458 |      2.522525291475752       | 2.545399333121626 |
| 1 |            australian dataset            | 8.70004995004995  | 8.587745587745587  | 8.616487678987678 | 8.671599234099233  | 8.650120712620712 |       8.38838245088245       | 8.65174408